# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

¡Empecemos!

In [2]:
url = 'https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=20&fecha1_anomes=2019-12&auAvailability=1&ida_vuelta=ida&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=20/12/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo='

In [3]:
from selenium import webdriver

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='../../chromedriver', options=options)
driver.get(url)

In [6]:
#Usaremos el Xpath para obtener la lista de vuelos
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')

In [7]:
vuelo = vuelos[0]

Obtenemos la información de la hora de salida, llegada y duración del vuelo

In [8]:
# Hora de salida
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

'19:20'

In [9]:
# Hora de llegada
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

'13:10'

In [10]:
# Duración del vuelo
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

'PT13H50M'

In [11]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
boton_escalas

<selenium.webdriver.remote.webelement.WebElement (session="ba78ed854c6b904e78c4c5c84ada931c", element="eea0ba2d-6b70-4b3b-954e-47465bf50e97")>

In [12]:
boton_escalas.click()

In [13]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="segments-graph"]/div[@class="segments-graph-segment"]')
segmentos

[<selenium.webdriver.remote.webelement.WebElement (session="ba78ed854c6b904e78c4c5c84ada931c", element="068f1279-48c2-4786-936c-c5237e4969d8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ba78ed854c6b904e78c4c5c84ada931c", element="6ef5a17c-9637-4d2c-ad9c-8e3d0daec6d2")>]

In [14]:
escalas = len(segmentos) - 1 #0 escalas si es un vuelo directo

# Clase 4
En esta clase obtendremos la información de las escalas que se encuentran en el modal que aparecen al clickear sobre el botón de escalas

In [15]:
segmento = segmentos[0]

In [17]:
# Origen
segmento.find_element_by_xpath('.//div[@class="departure"]/span[@class="ground-point-name"]').text

'EZE'

In [18]:
# Hora de salida
segmento.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

'19:20'

## RETO
Obtener:
- destino
- hora de llegada
- duración del vuelo
- duración de la escala. *Tip: el último segmento no tendrá esta información*
- número del vuelo
- modelo del avión

In [ ]:
# Destino
segmento.find_element_by_xpath('.//div[@class="arrival"]/span[@class="ground-point-name"]').text

In [ ]:
# Hora de llegada
segmento.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

In [ ]:
# Duración del vuelo
segmento.find_element_by_xpath('.//span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')

In [ ]:
# Numero del vuelo
segmento.find_element_by_xpath('.//span[@class="equipment-airline-number"]').text

In [ ]:
# Modelo de avion
segmento.find_element_by_xpath('.//span[@class="equipment-airline-material"]').text

In [ ]:
# Duracion de la escala
segmento.find_element_by_xpath('.//div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')

## CLASE
Una vez que hayamos obtenido toda la información, debemos cerrar el modal/pop-up.

In [19]:
driver.find_element_by_xpath('//div[@class="modal-dialog"]//button[@class="close"]').click()

Por último debemos obtener la información de las tarifas. Para eso, debemos clickear sobre el vuelo (sobre cualquier parte)

In [20]:
vuelo.click()

La información de los precios para cada tarifa está contenida en una tabla. Los precios en sí están en el footer y podemos sacar los nombres de la clase de cada elemento

In [21]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')

In [22]:
precios = []
for tarifa in tarifas:
    nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
    moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
    valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
    dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
    precios.append(dict_tarifa)
    print(dict_tarifa)

{'LIGHT': {'moneda': 'US$', 'valor': '1282,40'}}
{'PLUS': {'moneda': 'US$', 'valor': '1335,90'}}
{'TOP': {'moneda': 'US$', 'valor': '1773,50'}}


Será de gran utilidad armar funciones que resuelvan la extracción de información de cada sección de la página. Por eso te propongo que armes 3 funciones de las cuales te dejo las estructuras:

In [ ]:
def obtener_tiempos(vuelo):
    # Hora de salida
    salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    # Hora de llegada
    llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    # Duracion
    duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    return {'hora_salida': salida, 'hora_llegada': llegada, 'duracion': duracion}

## RETO
Armar funciones para obtener los datos de las escalas y las tarifas. 
Te dejo los prototipos:

In [ ]:
def obtener_precios(vuelo):
    tarifas = vuelo.find_elements_by_xpath(
        './/div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    precios = []
    for tarifa in tarifas:
        nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
        moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
        valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
        dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
        precios.append(dict_tarifa)
    return precios

In [ ]:
def obtener_datos_escalas(vuelo):
    segmentos = vuelo.find_elements_by_xpath('//div[@class="segments-graph"]/div[@class="segments-graph-segment"]')
    info_escalas = []
    for segmento in segmentos:
        # Origen
        origen = segmento.find_element_by_xpath(
            './/div[@class="departure"]/span[@class="ground-point-name"]').text
        # Hora de salida
        dep_time = segmento.find_element_by_xpath(
            './/div[@class="departure"]/time').get_attribute('datetime')
        # Destino
        destino = segmento.find_element_by_xpath(
            './/div[@class="arrival"]/span[@class="ground-point-name"]').text
        # Hora de llegada
        arr_time = segmento.find_element_by_xpath(
            './/div[@class="arrival"]/time').get_attribute('datetime')
        # Duración del vuelo
        duracion_vuelo = segmento.find_element_by_xpath(
            './/span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')
        # Numero del vuelo
        numero_vuelo = segmento.find_element_by_xpath(
            './/span[@class="equipment-airline-number"]').text
        # Modelo de avion
        modelo_avion = segmento.find_element_by_xpath(
            './/span[@class="equipment-airline-material"]').text
        # Duracion de la escala
        if segmento != segmentos[-1]:
            duracion_escala = segmento.find_element_by_xpath(
                './/div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')
        else:
            duracion_escala = ''

        # Armo un diccionario para almacenar los datos
        data_dict={'origen': origen, 
                    'dep_time': dep_time, 
                    'destino': destino,
                    'arr_time': arr_time,
                    'duracion_vuelo': duracion_vuelo,
                    'numero_vuelo': numero_vuelo,
                    'modelo_avion': modelo_avion,
                    'duracion_escala': duracion_escala}
        info_escalas.append(data_dict)
        
    return info_escalas

In [23]:
driver.close()